# Enumeration Algorithm

In [1]:
import numpy as np
import pandas as pd
import re

# Get joint distribution from Excel
I put my excel file in my assignment folder
You can change or add Random Vars without limitation but the Probability column name must be "P"

In [2]:
df = pd.read_excel (r'data.xlsx')
print (df)

    Season  Temp Weather     P
0   summer   hot     sun  0.35
1   summer   hot    rain  0.01
2   summer   hot     fog  0.01
3   summer   hot  meteor  0.00
4   summer  cold     sun  0.10
5   summer  cold    rain  0.05
6   summer  cold     fog  0.09
7   summer  cold  meteor  0.00
8   winter   hot     sun  0.10
9   winter   hot    rain  0.01
10  winter   hot     fog  0.02
11  winter   hot  meteor  0.00
12  winter  cold     sun  0.15
13  winter  cold    rain  0.20
14  winter  cold     fog  0.18
15  winter  cold  meteor  0.00


# Query input guide

your query must be like below
also you can add many evidences as column numbers

p(Weather|Season=winter,Temp=hot)

p(Weather|Season=winter)

p(Weather)

for speed of checking, i commented the input query from user
you can change the query or open user input query

In [3]:
#query = input("Enter Query: ") 
query = "p(Weather|Season=winter)";
print(query) 

p(Weather|Season=winter)


# Preprocessing
Here i seperate main random var and evidences with RegularExpression system.
you can pick comment to see seperation

In [4]:
params = re.findall(r'\((.*?)\)',query)[0].split('|')
#targetRV = targetRandomVariable
targetRV = params[0]
evidences = []
if len(params)>1:
    evidences = params[1].split(',')
#print("targetRV = " + targetRV)
#print(evidences)

# Evidence lines sellection - Step 1
select lines of data frame that there is evidence in it
you can see these lines prints below

In [5]:
list = df
for ev in evidences:
    ev = ev.split('=')
    evidenceRV = ev[0]
    evidenceVal = ev[1]
    list = list.where(df[evidenceRV] == evidenceVal)
print (list.dropna())

    Season  Temp Weather     P
8   winter   hot     sun  0.10
9   winter   hot    rain  0.01
10  winter   hot     fog  0.02
11  winter   hot  meteor  0.00
12  winter  cold     sun  0.15
13  winter  cold    rain  0.20
14  winter  cold     fog  0.18
15  winter  cold  meteor  0.00


# Marginal for Hidden Vars - Step2
find unique columns and do summation in probabilities

In [6]:
#marginal
RVsituations = list[targetRV]
RVsituations_arr = np.unique(RVsituations.dropna().to_numpy())
RVsituations_p_arr = []
for status in RVsituations_arr: 
    newlist = list['P'].where(df[targetRV] == status).dropna().to_numpy()
    newVal = np.sum(newlist);
    RVsituations_p_arr.append(newVal)

newdf = pd.DataFrame({targetRV:RVsituations_arr, 
                    'P':RVsituations_p_arr}) 
print(newdf)



    

  Weather     P
0     fog  0.20
1  meteor  0.00
2    rain  0.21
3     sun  0.25


# Normalization for convert joint to conditional - step3
find Alpha parameter and Multiply with each lines

In [7]:
alpha = np.sum(RVsituations_p_arr)

temp_arr = []
for status in RVsituations_p_arr:
    newVal = status/alpha
    temp_arr.append(newVal)
RVsituations_p_arr = temp_arr  

newdf = pd.DataFrame({targetRV:RVsituations_arr, 
                    'P':RVsituations_p_arr}) 
print(newdf)

  Weather         P
0     fog  0.303030
1  meteor  0.000000
2    rain  0.318182
3     sun  0.378788


# Get total summery of probabilities

In [8]:
finalP = np.sum(RVsituations_p_arr)
#print(query + " = " + finalP)
print(query + " = " + str(finalP))

p(Weather|Season=winter) = 0.9999999999999999
